In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from numba import jit
from random import random
from scipy.special import expit

In [ ]:
class networkBaseClass(object):
    def __init__(self):
        self.size = 1
        self.olscores = np.array([])
    
    
    def setNullNetwork(self):
        self.network=[]
        for i in range(len(self.layersizes)-2):
            self.network.append(np.zeros([self.layersizes[i], self.layersizes[i+1]]))
            self.network.append(np.zeros([self.layersizes[i+1]]))
        self.network.append(np.zeros([self.layersizes[i+1], self.layersizes[i+2]]))
    
    @jit
    def prop(self, val):
        for i in range(0,len(self.network)-2,2):
            #RelU
            val = np.maximum(0,np.dot(val,self.network[i]) - self.network[i+1])
            #-1 to 1 ReLU
            #val = np.minimum(1,np.maximum(-1,np.dot(val,self.network[i]) - self.network[i+1]))
            #Sigmoid
            #val = expit(np.dot(val,self.network[i]) - self.network[i+1])
        return np.dot(val,self.network[-2])
    
    @jit
    def cost(self, network):
        cost = 0
        for i in range(len(self.data[0])):
            cost += (nnStuff.prop(network, self.data[0][i])-self.data[1][i])**2
        return cost    
    
    
    def vecMain(self, size):
        types = nnStuff.netToVec([i+len(np.shape(i)) for i in self.network])
        score = [self.cost(self.network), None]
        self.size = size
        vecNet = nnStuff.netToVec(self.network)
        for i in range(len(nnStuff.netToVec(self.network))):
            vecNet[i] += self.size
            c = self.cost(nnStuff.vecToNet(vecNet, self.layersizes))
            if c < score[0]:
                score = [c, i]
            vecNet[i] -= self.size*2
            c = self.cost(nnStuff.vecToNet(vecNet, self.layersizes))
            if c < score[0]:
                score = [c, -i-1]
            vecNet[i] += self.size
        if score[1] != None:
            if vecNet[score[1]] >= 0:
                vecNet[score[1]] += self.size
            else:
                vecNet[-score[1]-1] -= self.size
            self.network =  nnStuff.vecToNet(vecNet, self.layersizes)
        else:
            return False
            
    
    def show(self):
        fig = plt.figure()
        ax = fig.gca()
        ax.plot(self.data[0], self.data[1])
        ax.plot(self.data[0], list([self.prop([i]).flatten() for i in self.data[0]]))
        plt.show()
        plt.close(fig)
    
    
    def showData(self, dat):
        fig = plt.figure()
        ax = fig.gca()
        if max(dat) >= max(self.data[0]) and min(dat) <= min(self.data[0]):
            ax.plot(self.data[0], self.data[1])
        ax.plot(dat, list([self.prop(i).flatten() for i in dat]))
        plt.show()
        plt.close(fig)
    
class nnStuff:
    @jit
    def prop(network, val):
        for i in range(0,len(network)-2,2):
                #val = np.maximum(0,np.dot(val,network[i]) - network[i+1])
                val = expit(np.dot(val,network[i]) - network[i+1])
        return np.dot(val,network[-2])
    
    
    def getNullNetwork(layersizes):
        nn=[]
        for i in range(len(layersizes)-2):
            nn.append(np.zeros([layersizes[i], layersizes[i+1]]))
            nn.append(np.zeros([layersizes[i+1]]))
        nn.append(np.zeros([layersizes[i+1], layersizes[i+2]]))
        return nn
    
    @jit
    def netToVec(network):
        vec = np.array([])
        for i in network:
            vec = np.append(vec, i.flatten())
        return vec

    
    def vecToNet(vec, layersizes):
        index = 0
        network = nnStuff.getNullNetwork(layersizes)
        for i in range(len(layersizes)-2):
            for j in range(layersizes[i]):
                for k in range(layersizes[i+1]):
                    network[2*i][j][k] = vec[index]
                    index += 1
            for j in range(layersizes[i+1]):
                network[2*i+1][j] = vec[index]
                index += 1
        for j in range(layersizes[i+1]):
            for k in range(layersizes[i+2]):
                network[2*i+2][j][k] = vec[index]
                index += 1
        return network
    
    def randomize(network, layersizes):
        net = nnStuff.netToVec(network)
        rand = np.random.randint(-100,100,len(net))/100
        return nnStuff.vecToNet(net+rand, layersizes)

In [ ]:
class sineBot(networkBaseClass):
    layersizes = [1,10,10,1]
    inp = [i/20 for i in range(0,21)]
    outp = [np.sin(i*2*np.pi)**2 for i in inp]
    data = [[i for i in inp],[i for i in outp]]
    transData = [i for i in zip(*data)]
    
    def __init__(self):
        super(sineBot, self).setNullNetwork()
        super(sineBot, self).__init__()

In [ ]:
s = sineBot()

In [ ]:
s.network = nnStuff.vecToNet(np.random.rand(len(nnStuff.netToVec(s.network))), s.layersizes)

In [ ]:
for i in range(101):
    a=s.vecMain(1)
    if a == False:
        print(":(")
    if i*10 % 100 == 0:
        print(i)

In [ ]:
s.show()

In [ ]:
s.network